In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
import cartopy.feature as feature

In [ ]:
#Make an empty plot with default projection and coastlines
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

In [ ]:
file='/home/pdirmeye/classes/clim680_2022/OISSTv2/monthly/sst.mnmean.nc'
ds=xr.open_dataset(file)
ds

## Calculate seasonal means

In [ ]:
ds_seas=ds.groupby('time.season').mean()
ds_seas

### Plot all 4 seasons on the same figure in a 2x2 configuration

In [ ]:
plt.subplot(2,2,1)
plt.contourf(ds_seas['sst'].sel(season='DJF'))
plt.title('DJF')
plt.subplot(2,2,2)
plt.contourf(ds_seas['sst'].sel(season='MAM'))
plt.title('MAM')
plt.subplot(2,2,3)
plt.contourf(ds_seas['sst'].sel(season='JJA'))
plt.title('JJA')
plt.subplot(2,2,4)
plt.contourf(ds_seas['sst'].sel(season='SON'))
plt.title('SON')
plt.colorbar()
plt.suptitle('SST (degC)')

This was not a very efficient way to plot the four panels, but it worked.

We could use a loop to make it more efficient. 
Also, there is the problem that the color bar has caused the last panel to be compressed.
We will deal with that later.

First, let's use a loop to plot the panels for each season, and choose colors that seem more appropriate for temperature.

In [ ]:
clevs=np.arange(-5,36,5)
for i,seas in enumerate(ds_seas['season']):
    plt.subplot(2,2,i+1)
    plt.contourf(ds_seas['sst'].sel(season=seas),levels=clevs,cmap='RdBu_r')
    plt.title(seas.values)
    plt.colorbar()
plt.suptitle('SST (deg C)')

### What if we want to plot on a map projection?  
Its a little more complicated.
We have to use `plt.subplots`

In [ ]:
# Define the figure and each axis for the 2 rows and 2 columns
fig, axs = plt.subplots(nrows=2,ncols=2,
                        subplot_kw={'projection': ccrs.PlateCarree()},
                        figsize=(11,5.5))

# axs is a 2 dimensional array of `GeoAxes`.  
# We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the seasons and plot
for i,seas in enumerate(ds_seas['season']):

        # Select the season
        data=ds_seas['sst'].sel(season=seas)

        # Add the cyclic point
        data,lons=add_cyclic_point(data,coord=ds_seas['lon'])

        # Contour plot
        cs=axs[i].contourf(lons,ds_seas['lat'],data,clevs,
                          transform = ccrs.PlateCarree(),
                          cmap='coolwarm',extend='both')

        # Title each subplot with the name of the model
        axs[i].set_title(seas.values)

        # Draw the coastines for each subplot
        axs[i].coastlines()

### Now lets make it look nicer 
* adding a single colorbar 
* add a big title
* label our lats and lons

In [ ]:
# Define the figure and each axis for the 2 rows and 2 columns
fig, axs = plt.subplots(nrows=2,ncols=2,
                        subplot_kw={'projection': ccrs.PlateCarree()},
                        figsize=(11,8.5))

# axs is a 2 dimensional array of `GeoAxes`.  
# We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the seasons and plot
for i,seas in enumerate(ds_seas['season']):

        # Select the season
        data=ds_seas['sst'].sel(season=seas)

        # Add the cyclic point
        data,lons=add_cyclic_point(data,coord=ds_seas['lon'])

        # Contour plot
        cs=axs[i].contourf(lons,ds_seas['lat'],data,clevs,
                          transform = ccrs.PlateCarree(),
                          cmap='coolwarm',extend='both')

       # Longitude labels
        axs[i].set_xticks(np.arange(-180,181,60), crs=ccrs.PlateCarree())
        lon_formatter = cticker.LongitudeFormatter()
        axs[i].xaxis.set_major_formatter(lon_formatter)

        # Latitude labels
        axs[i].set_yticks(np.arange(-90,91,30), crs=ccrs.PlateCarree())
        lat_formatter = cticker.LatitudeFormatter()
        axs[i].yaxis.set_major_formatter(lat_formatter)

        # Title each subplot with the name of the season
        axs[i].set_title(seas.values)

        # Draw the coastines for each subplot
        axs[i].coastlines()
        
# Adjust the location of the subplots 
# on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.05, right=0.95,
                    wspace=0.2, hspace=0.25)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.175, 0.6, 0.02])

# Draw the colorbar
cbar=fig.colorbar(cs, cax=cbar_ax,orientation='horizontal',label='˚C')

# Add a big title at the top
plt.suptitle('Sea Surface Temperature from OISSTv3',fontsize=20)

### Faceting
Faceting allows you to plot subplots across the dimensios of an `xarray.Dataset` using many fewer lines of code. It uses the metadata to default label the subplots for you and makes a single colorbar for all subplots.
Take a look at the [documentation](https://docs.xarray.dev/en/stable/user-guide/plotting.html#plotting-faceting).

In [ ]:
fig = plt.figure(figsize=(11,8.5))

fg = ds_seas['sst'].plot(x='lon',y='lat',col='season',col_wrap=2,
    levels=clevs,cmap='RdBu_r',extend='both',
    subplot_kws={"projection": ccrs.PlateCarree()},
    cbar_kwargs={
        "orientation": "horizontal",
        "shrink": 0.8,
        "aspect": 40,
        "label": "Temperature [˚C]",
    },)

for ax in fg.axes.flat:
    ax.coastlines()
    ax.add_feature(feature.LAND, zorder=2, color='#D5DCC9')
    